In [73]:
from urllib.request import urlopen,Request
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import matplotlib.pyplot as plt
#in the below code we extract the data using beautiful soup

In [74]:
finvz_url='https://finviz.com/quote.ashx?t='
tickers=['AMZN','MS']
news_tables={}

In [75]:
for ticker in tickers:
    url = finvz_url+ticker;
    
    request=Request(url=url,headers={'user-agent':'my-app'})
    response=urlopen(request)
    #print(response) used to this test
    html = BeautifulSoup(response,'html')
    #print(html)
    news_table=html.find(id='news-table')
    news_tables[ticker]=news_table;
    

In [69]:
print(news_tables)#we are trying to map each ticker with it's news articles which are in the from of tables

{'AMZN': <table border="0" cellpadding="1" cellspacing="0" class="fullview-news-outer" id="news-table" width="100%">
<tr class="cursor-pointer" onclick="trackAndOpenNews(event, 'Motley Fool', 'https://finance.yahoo.com/m/8c5c2db2-54fd-3a2a-a12c-06eaf0055e09/3-top-e-commerce-stocks-to.html');">
<td align="right" width="130">
            Jul-27-23 07:40AM
        </td>
<td align="left">
<div class="news-link-container">
<div class="news-link-left">
<a class="tab-link-news" href="https://finance.yahoo.com/m/8c5c2db2-54fd-3a2a-a12c-06eaf0055e09/3-top-e-commerce-stocks-to.html" rel="nofollow" target="_blank">3 Top E-Commerce Stocks to Buy Right Now</a>
</div>
<div class="news-link-right">
<span>(Motley Fool)</span></div></div></td></tr>
<tr class="cursor-pointer" onclick="trackAndOpenNews(event, 'Motley Fool', 'https://finance.yahoo.com/m/1307cfe6-8f4e-3b8f-a00f-d67c3283af89/2-top-buffett-stocks-to-buy.html');">
<td align="right" width="130">
            07:11AM
        </td>
<td align="lef

In [81]:
#in the next steps we try to manipulate data and scrap what we need from it
#we need the title and timestamps from it so we'll be scarpping that
""""amzn_data=news_tables['AMZN']
amzn_rows=amzn_data.findAll('tr')#all tr elements will be returned
for idx,row in enumerate(amzn_rows):
    title=row.a.text#anchor tag's text
    timestamp=row.td.text
    print(timestamp + " "+title)
    """
parsed_data=[]#this list will contain the ticker the time date and title

for ticker,news_table in news_tables.items():
    for row in news_table.findAll('tr'):
        title = row.a.text
        timestamp= row.td.text
        date_data=timestamp.split(' ')#we might have 2 componenets so we are spliting it with space to get both
        
        if len(date_data)==1:
            time=date_data[0]
        else:
            date = date_data[0]
            time = date_data[1]
        parsed_data.append([ticker,date,time,title])

print(parsed_data)
    

[['AMZN', '\r\n', '', '3 Top E-Commerce Stocks to Buy Right Now'], ['AMZN', '\r\n', '', '2 Top Buffett Stocks to Buy and Hold for the Long Haul'], ['AMZN', '\r\n', '', 'Got $3,000? 3 Growth Stocks to Double Up on Right Now'], ['AMZN', '\r\n', '', 'Stocks Higher, Dovish Fed Rate Hike, Meta Surges, Amazon Earnings Next, Intel On Deck - 5 Things To Know'], ['AMZN', '\r\n', '', 'If you think drug addiction isnt a problem in your workplace, youre wrong'], ['AMZN', '\r\n', '', '3 "Magnificent Seven" Stocks to Buy Now and 1 to Avoid, According to Wall Street'], ['AMZN', '\r\n', '', 'Zacks Investment Ideas feature highlights: Apple, Palo Alto Networks, Microsoft, Fidelity Contra and Amazon'], ['AMZN', '\r\n', '', 'Amazon Debuts HIPAA-Eligible Generative AI for Healthcare Providers'], ['AMZN', '\r\n', '', 'EBay Drops as Quarterly Profit Outlook Misses Estimates'], ['AMZN', '\r\n', '', '10 Stocks That Could Be the Next Apple or Amazon'], ['AMZN', '\r\n', '', '5 Amazing Amazon Buys You Can Find T

In [82]:
#pandas allowws us to apply function easily
import nltk.sentiment.vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader=SentimentIntensityAnalyzer()
df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])
f=lambda title:vader.polarity_scores(title)['compound']
df['compound']=df['title'].apply(f)
df['date'] = pd.to_datetime(df.date).dt.date

In [ ]:
plt.figure(figsize=(10,8))
mean_df = df.groupby(['ticker', 'date']).mean().unstack()
mean_df = mean_df.xs('compound', axis="columns")
mean_df.plot(kind='bar')
plt.show()